# MFT Asynchronous QC

In [ ]:
_period = "LHC22q"
_pass0 = "online"
_pass1 = "apass1"
_pass2 = "apass2"
_pass3 = "apass3"
_pass4 = "none"
_run = 529009

In [ ]:
import ROOT
from ROOT import gStyle, gPad
%jsroot on

# print the current configuration
print(f"period: {_period}")
print(f"run:    {_run}")
print(f"pass0:  {_pass0}")
print(f"pass1:  {_pass1}")
print(f"pass2:  {_pass2}")
print(f"pass3:  {_pass3}")
print(f"pass4:  {_pass4}")

nPasses = 0
passes = [_pass0,_pass1,_pass2,_pass3,_pass4]
files = ROOT.TObjArray(5)
files.SetOwner()
for i in range(0,5):
    f = ROOT.TFile()
    if passes[i] != "none":
        nPasses+=1
        f = ROOT.TFile.Open(f"Results/{_period}/runsRootFiles/{_run}_{passes[i]}.root","read")
    files.AddAt(f,i)
    
colors = [ROOT.kGray+1,ROOT.kBlue,ROOT.kRed,ROOT.kGreen+1,ROOT.kYellow+1] # five colors, one for each pass

names = [
# track position (6 in total)
"mMFTTrackEtaPhi_5_MinClusters",
"mMFTTrackXY_5_MinClusters",
"mMFTTrackEtaPhi_7_MinClusters",
"mMFTTrackXY_7_MinClusters",
"mMFTTrackEtaPhi_8_MinClusters",
"mMFTTrackXY_8_MinClusters",
# tracks (11 in total)
"mMFTTrackEta",
"mMFTTrackNumberOfClusters",
"mMFTTrackPhi",
"mMFTTrackTanl",
"mNOfTracksTime",
"mMFTTrackInvQPt",
"mMFTCATrackPt",
"mMFTLTFTrackPt",
"mMFTCATrackEta",
"mMFTLTFTrackEta",
"mMFTTracksBC",
# clusters (4 in total)
"mMFTClusterPatternIndex",
"mMFTClusterSensorIndex",
"mMFTClustersROFSize",
"mNOfClustersTime"]

# which histograms to plot with log scale on the y-axis
logScales = [
"mMFTCATrackPt",
"mMFTLTFTrackPt",
"mMFTClusterPatternIndex",
"mMFTClusterSensorIndex",
"mMFTClustersROFSize"]

# which histograms to plot with the ratio panel
withRatios = [
"mMFTTrackEta",
"mMFTTrackNumberOfClusters",
"mMFTTrackPhi",
"mMFTTrackTanl",
"mMFTTrackInvQPt",
"mMFTCATrackPt",
"mMFTLTFTrackPt",
"mMFTCATrackEta",
"mMFTLTFTrackEta"]

gStyle.SetTitleX(0.45)
gStyle.SetTitleY(1.0)
gStyle.SetOptStat(0)
gStyle.SetOptTitle(0);

def createArrOfLegends(length,xMin,yMin,xMax,yMax):
    arr = ROOT.TObjArray(length)
    arr.SetOwner()
    for i in range(length):
        l = ROOT.TLegend(xMin,yMin,xMax,yMax)
        #l.SetBorderSize(0)
        l.SetFillColor(0)
        #l.SetFillStyle(0)
        l.SetMargin(0.23)
        l.SetTextFont(63)
        l.SetTextSize(0.08)
        arr.AddAt(l,i)
    return arr

arrLeg1DTop = createArrOfLegends(15,0.79,0.85,0.98,1.00)
arrLeg1DBottom = createArrOfLegends(15,0.83,0.76,0.98,1.00)
arrLeg2D = createArrOfLegends(6*nPasses,0.60,0.90,0.95,1.00)

hRatios = ROOT.TObjArray(15*nPasses)
hRatios.SetOwner()
lines = ROOT.TObjArray(15)
lines.SetOwner()

ltx = ROOT.TLatex()
ltx.SetTextFont(63)
ltx.SetTextSize(13)
ltx.SetTextAlign(11)
ltx.SetNDC()

def createPad(name, xMin, yMin, xMax, yMax):
    p = ROOT.TPad(name,"",xMin,yMin,xMax,yMax)
    p.SetRightMargin(0.05)
    p.SetLeftMargin(0.12)
    return p

def setHisto(h):
    # x-axis
    h.GetXaxis().SetTitleFont(63)
    h.GetXaxis().SetTitleSize(14)
    h.GetXaxis().SetLabelFont(63)
    h.GetXaxis().SetLabelSize(14)
    # y-axis
    h.GetYaxis().SetMaxDigits(3)
    h.GetYaxis().SetTitleFont(63)
    h.GetYaxis().SetTitleSize(14)
    h.GetYaxis().SetLabelFont(63)
    h.GetYaxis().SetLabelSize(14)
    return h

def plotComparison1D(canvasName, drawOption, nCols, nRows, histoNames, idxFirst):
    c = ROOT.TCanvas(canvasName,"",nCols*450,nRows*400)
    c.Divide(nCols,nRows)
    i = 1
    # create canvas for each histogram and plot all passes
    for histoName in histoNames:
        title = ""
        normalizations = [0, 0, 0, 0, 0]
        maximums = [0, 0, 0, 0, 0]
        # x-axis settings for the ratio panel
        xMin = 1e10
        xMax = 0.
        xTitle = ""
        # loop over passes
        for j in range(0,5):
            if passes[j] != "none":
                h = files.At(j).Get(histoName)
                if h != None:
                    xMin = h.GetBinLowEdge(1)
                    xMax = h.GetBinLowEdge(h.GetNbinsX()+1)
                    xTitle = h.GetXaxis().GetTitle()
                    title = h.GetTitle()
                    normalizations[j] = h.Integral()
                    if(normalizations[j] > 0):
                        h.Scale(1./normalizations[j])
                    maximums[j] = h.GetMaximum()
        maximum = max(maximums[0:5])
        c.cd(i) 
        # does this histogram need to be plotted in log scale?
        logScale = False
        for name in logScales:
            if histoName == name:
                logScale = True
        # is the histogram to be plotted with the ratio panel or not
        plotWithRatios = False
        for name in withRatios:
            if histoName == name:
                plotWithRatios = True
        if plotWithRatios == True:
            # top panel -> distribution
            pTop = createPad(f"pTop_{i}",0.0,0.35,1.0,1.0)
            pTop.SetTopMargin(0.135)
            pTop.SetBottomMargin(0)
            pTop.Draw()
            pTop.cd()
            if logScale == True:
                pTop.SetLogy()
            # load the legends
            lTop = arrLeg1DTop.At(idxFirst)
            lBottom = arrLeg1DBottom.At(idxFirst)
            # loop over passes
            for j in range(0,5):
                if passes[j] != "none":
                    h = files.At(j).Get(histoName)
                    if h != None:
                        lTop.AddEntry(h,f"{passes[j]}: {normalizations[j]:.2e}")
                        if(normalizations[j] > 0):
                            h.Scale(1./normalizations[j])
                        h = setHisto(h)
                        h.GetYaxis().SetTitle("# entries (norm. to 1.0)")
                        h.SetLineColor(colors[j])
                        h.SetLineWidth(2)
                        if logScale == False:
                            h.GetYaxis().SetRangeUser(0.,maximum*1.05)
                        h.Draw(f"{drawOption} same")
            lTop.Draw()
            ltx.DrawLatex(0.25,0.940,f"{_period}, {_run}:")
            ltx.DrawLatex(0.25,0.875,f"{title}")
            # bottom panel -> ratios
            c.cd(i) 
            pBottom = createPad(f"pBottom_{i}",0.0,0.0,1.0,0.35)
            pBottom.SetTopMargin(0)
            pBottom.SetBottomMargin(0.25)
            pBottom.Draw()
            pBottom.cd()
            hAxisBottom = gPad.DrawFrame(xMin,0.5,xMax,1.7)
            setHisto(hAxisBottom)
            hAxisBottom.GetYaxis().SetTitle("ratio")
            hAxisBottom.GetXaxis().SetTitle(xTitle)
            hAxisBottom.Draw("AXIS")
            #ratios: apass2/apass1, 3/1, 3/2
            idxPass0 = [2, 3, 3] # apass number to be divided
            idxPass1 = [1, 1, 2] # with data from here
            line = ROOT.TLine(xMin,1.0,xMax,1.0)
            line.SetLineColor(ROOT.kBlack)
            line.SetLineWidth(1)
            line.SetLineStyle(2)
            lines.AddAt(line,idxFirst)
            for j in range(0,3): 
                if passes[idxPass0[j]] != "none" and passes[idxPass1[j]] != "none":
                    h0 = files.At(idxPass0[j]).Get(histoName)
                    if(normalizations[idxPass0[j]] > 0):
                        h0.Scale(1./normalizations[idxPass0[j]])
                    h1 = files.At(idxPass1[j]).Get(histoName)
                    if(normalizations[idxPass1[j]] > 0):
                        h1.Scale(1./normalizations[idxPass1[j]])
                    hRatio = ROOT.TH1F()
                    hRatio = h0.Clone(f"{histoName}_{idxPass0[j]}/{idxPass1[j]}")
                    hRatio.Divide(h1)
                    hRatio.SetLineWidth(2)
                    hRatio.SetLineStyle(9)
                    hRatio.SetLineColor(colors[j+1])
                    hRatios.AddAt(hRatio,idxFirst*nPasses+j)
                    lBottom.AddEntry(hRatio,f"{passes[idxPass0[j]]}/{passes[idxPass1[j]]}")
                    hRatio.Draw(f"{drawOption} same")
            line.Draw("same")
            lBottom.Draw()
        else:
            # only one panel:
            gPad.SetTopMargin(0.090)
            gPad.SetLeftMargin(0.12)
            gPad.SetRightMargin(0.08)
            # load the legends
            lTop = arrLeg1DTop.At(idxFirst)
            if logScale == True:
                c.cd(i).SetLogy()
            for j in range(0,5):
                if passes[j] != "none":
                    h = files.At(j).Get(histoName)
                    if h != None:
                        lTop.AddEntry(h,f"{passes[j]}: {normalizations[j]:.2e}")
                        if(normalizations[j] > 0):
                            h.Scale(1./normalizations[j])
                        h = setHisto(h)
                        h.GetYaxis().SetTitle("# entries (norm. to 1.0)")
                        h.SetLineColor(colors[j])
                        h.SetLineWidth(2)
                        if logScale == False:
                            h.GetYaxis().SetRangeUser(0.,maximum*1.05)
                        h.Draw(f"{drawOption} same")
            lTop.Draw()
            ltx.DrawLatex(0.25,0.955,f"{_period}, {_run}:")
            ltx.DrawLatex(0.25,0.920,f"{title}")
        i+=1
        idxFirst+=1
    return c

## TRACK POSITION:

In [ ]:
nCols = nPasses
nRows = 6
c = ROOT.TCanvas("trackPos","",nCols*400,nRows*300)
c.Divide(nCols,nRows)
empty = ROOT.TObject()
i = 1
for histoName in names[0:6]:
    for j in range(0,nPasses):
        idxCvs = nPasses*(i-1)+j+1
        c.cd(idxCvs)
        idxPass = nPasses-j-1
        gPad.SetTopMargin(0.08)
        gPad.SetRightMargin(0.14)
        gPad.SetBottomMargin(0.12)
        if passes[idxPass] != "none":
            h = files.At(idxPass).Get(histoName)
            if h != None:
                h = setHisto(h)
                h.GetZaxis().SetMaxDigits(3)
                h.GetZaxis().SetLabelFont(63)
                h.GetZaxis().SetLabelSize(14)
                h.GetYaxis().SetTitleOffset(0.85)
                h.Draw("colz")
                ltx.DrawLatex(0.10,0.95,f"{h.GetTitle()}, {passes[idxPass]}: {h.Integral():.2e}")
    i+=1
c.Draw()

## TRACK PROPERTIES:

In [ ]:
c2 = plotComparison1D("trackProp","hist",2,6,names[6:17],0)
c2.Draw()

## CLUSTER PROPERTIES:

In [ ]:
c3 = plotComparison1D("clusterProp","hist",2,2,names[17:21],11)
c3.Draw()